In [1]:
import urllib2
from bs4 import BeautifulSoup as Soup
from lxml import etree
import time
import pandas as pd

def get_all_subjects(url):
    res = []
    res += get_subjects(url)
    while has_next_page(url):
        new_url = url[0:len(url)-1] + str(int(url[len(url)-1])+1)
        res += get_subjects(new_url)
        url = new_url
    return res
        
def has_next_page(url):
    page = urllib2.urlopen(url)
    soup = Soup(page)
    if soup.find_all('a', {"href": url[0:len(url)-1] + str(int(url[len(url)-1])+1)}):
        return True
    else:
        return False

def get_subjects(url):
    page = urllib2.urlopen(url)
    soup = Soup(page)
    sections = soup.find_all('ul', {"class": "plain margin7"})
    res = []
    for section in sections:
        for listelement in section.findAll('li'):
            res.append(listelement.get_text())
    return res
    
def find_bill_subjects(billName):
    if billName[0:2] == "PN":
        return []
    else:
        bill = billName.split()
        if bill[0] == 'H.R.':
            url = "https://www.congress.gov/bill/114th-congress/house-bill/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
        elif bill[0] == 'S.':
            url = "https://www.congress.gov/bill/114th-congress/senate-bill/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
        elif bill[0] == 'S.J.Res.':
            url = "https://www.congress.gov/bill/114th-congress/senate-joint-resolution/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
        elif bill[0] == 'H.J.Res.':
            url = "https://www.congress.gov/bill/114th-congress/house-joint-resolution/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
        elif bill[0] == 'S.Con.Res.':
            url = "https://www.congress.gov/bill/114th-congress/house-concurrent-resolution/" + bill[1] + "/subjects?page=1"
            return get_all_subjects(url)
            
print find_bill_subjects("S.Con.Res. 11")

[u'Blood and blood diseases', u'Commemorative events and holidays', u'Congressional tributes', u'Health', u'Organ and tissue donation and transplantation', u'Social work, volunteer service, charitable organizations']


In [9]:
#339 is the max number for this session
tree = etree.parse('http://www.senate.gov/legislative/LIS/roll_call_votes/vote1141/vote_114_1_00001.xml')

vote_date = tree.findall("vote_date")[0].text
vote_result = tree.findall("vote_result")[0].text
vote_title = tree.findall("document")[0][3].text
if vote_title:
    subjects = find_bill_subjects(vote_title)
elif not vote_title:
    vote_title = "Amendment to " + tree.findall("amendment")[0][3].text
    subjects = find_bill_subjects(tree.findall("amendment")[0][3].text)
    if not vote_title:
        subjects = find_bill_subjects("PN")
        vote_title = "Bro I have no idea what this about"

vote_dict = {'voteDate': [vote_date], 'title': [vote_title], 'voteResult': [vote_result], 'subjects': [subjects]}
for senator in tree.getiterator("member"):
    vote_dict[senator[2].text + " " + senator[0].text] = [senator[5].text]
    
#initializes dictionary

for i in range(2, 340):
    if i % 10 == 0:
        print i
    url = "http://www.senate.gov/legislative/LIS/roll_call_votes/vote1141/vote_114_1_" + '{0:05d}'.format(i) + '.xml'
    tree = etree.parse(url)
    vote_date = tree.findall("vote_date")[0].text
    vote_result = tree.findall("vote_result")[0].text
    vote_title = tree.findall("document")[0][3].text
    if vote_title:
        subjects = find_bill_subjects(vote_title)
    elif not vote_title:
        vote_title = "Amendment to " + tree.findall("amendment")[0][3].text
        subjects = find_bill_subjects(tree.findall("amendment")[0][3].text)
        if not vote_title:
            subjects = find_bill_subjects("PN")
            vote_title = "Bro I have no idea what this about"
    vote_dict['voteDate'].append(vote_date)
    vote_dict['title'].append(vote_title)
    vote_dict['voteResult'].append(vote_result)
    vote_dict['subjects'].append(subjects)
    for senator in tree.getiterator("member"):
        vote_dict[senator[2].text + " " + senator[0].text].append(senator[5].text)
    time.sleep(1)
    
print len(vote_dict['subjects'])

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
339


In [10]:
df = pd.DataFrame(data=vote_dict)
df.to_csv('rollCallVotes_iter3.csv' index=False)